In [12]:
import pandas as pd
import json


In [10]:
df_games = pd.read_csv('data/games.csv')
df_games.head()


,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
0,10090,Call of Duty: World at War,2008-11-18,True,False,False,Very Positive,92,37039,19.99,19.99,0.0,True
1,13500,Prince of Persia: Warrior Within™,2008-11-21,True,False,False,Very Positive,84,2199,9.99,9.99,0.0,True
2,22364,BRINK: Agents of Change,2011-08-03,True,False,False,Positive,85,21,2.99,2.99,0.0,True
3,113020,Monaco: What's Yours Is Mine,2013-04-24,True,True,True,Very Positive,92,3722,14.99,14.99,0.0,True
4,226560,Escape Dead Island,2014-11-18,True,False,False,Mixed,61,873,14.99,14.99,0.0,True


In [11]:
df_gen_users = pd.read_csv('data/generated_users.csv')
df_gen_users.head()

,steam_id,appids
0,76561198012345678,"570,730,440,550,271590"
1,76561198023456789,"1085660,381210,105600,252950,620"
2,76561198034567890,"230410,292030,1172620,582010,359550"
3,76561198045678901,"413150,294100,227300,435150,386360"
4,76561198056789012,"346110,377160,221100,252490,239140"


In [29]:

def extract_unique_tags(game_metadata_list):
    unique_tags = set()
    for game_metadata in game_metadata_list:
        tags = game_metadata.get("tags", [])
        unique_tags.update(tags)
    return list(unique_tags)


def write_tags_to_file(tags, filename):
    with open(filename, 'w') as f:
        for tag in tags:
            f.write(f"{tag}\n")

# Read game metadata JSON objects from a file
with open('data/games_metadata2.json', 'r') as f:
    game_metadata_jsons = [json.loads(line.strip()) for line in f if line.strip()]

# Read game metadata JSON objects from a file
# jsonFile = open('games_metadata.json', 'r')
# game_metadata_jsons = json.load(jsonFile)
# jsonFile.close()

# Extract unique tags
unique_tags = extract_unique_tags(game_metadata_jsons)
print("unique_tags", unique_tags)

# Write unique tags to a file
write_tags_to_file(unique_tags, 'unique_tags.txt')

# Create a DataFrame with unique tags as columns
df_game_tags_binary = pd.DataFrame(columns=unique_tags)

for game_metadata in game_metadata_jsons:
    game_tags = game_metadata.get("tags", [])
    
    binary_tags = []
    for tag in unique_tags:
        if tag in game_tags:
            binary_tags.append(1)
        else:
            binary_tags.append(0)
    
    app_id = game_metadata['app_id']
    df_game_tags_binary.loc[app_id] = binary_tags

df_game_tags_binary.head()
# Save the DataFrame to a CSV file
df_game_tags_binary.to_csv('game_tags_binary.csv')


unique_tags ['Loot', 'Adventure', 'Great Soundtrack', 'Strategy', 'Co-op', 'First-Person', 'Platformer', 'War', 'Online Co-Op', 'Third Person', 'FPS', 'Gore', 'Horror', 'Moddable', 'Multiplayer', 'Atmospheric', 'Dark Fantasy', 'Top-Down', 'Third-Person Shooter', 'Action', 'Shooter', 'Heist', 'Classic', 'Survival Horror', 'Dark', 'Indie', 'Tanks', 'Singleplayer', 'Local Co-Op', 'Stealth', 'Hack and Slash', 'Time Travel', '2D', 'Historical', 'World War II', 'Open World', 'Zombies', 'Cartoony', 'Casual', 'Local Multiplayer', 'Controller', 'Time Manipulation', 'Puzzle', 'Parkour', 'Survival', 'Crime', 'Fantasy', 'Anime', 'Story Rich', 'Pixel Graphics']


,Loot,Adventure,Great Soundtrack,Strategy,Co-op,First-Person,Platformer,War,Online Co-Op,Third Person,...,Controller,Time Manipulation,Puzzle,Parkour,Survival,Crime,Fantasy,Anime,Story Rich,Pixel Graphics
10090,0,1,1,0,1,1,0,1,1,0,...,0,0,0,0,1,0,0,0,0,0
13500,0,1,1,0,0,0,1,0,0,1,...,1,1,1,1,0,0,1,0,1,0
22364,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
113020,1,1,1,1,1,0,0,0,1,0,...,1,0,0,0,0,1,0,0,0,1
226560,0,1,0,0,1,0,0,0,0,1,...,0,0,0,0,1,0,0,1,0,0
